In [ ]:
using LoopVectorization
using SharedArrays
using BenchmarkTools

# Preperare distributed
using Distributed
addprocs(Sys.CPU_THREADS - nprocs())
print("Number of workers: ", nprocs(), "\n")


@everywhere using Plots

@everywhere begin
    ENV["GKSwstype"] = "nul"  # Headless mode - no display
end

# Problem 1.1A & B

In [ ]:
# Setup for benchmarking
N_test = 100000
psi_tst = rand(N_test)


In [ ]:
"""
    wave_equation(psi, c)

Compute the second time derivative of the wave function using the 1D wave equation.

Uses the finite difference approximation of the wave equation: ∂²ψ/∂t² = c² ∂²ψ/∂x²

# Arguments
- `psi`: Vector of wave function values at discrete spatial points
- `c`: Wave speed (propagation velocity)

# Returns
- `d2psi_dt2`: Vector of second time derivatives at interior points (length = length(psi) - 2)

# Notes
- Uses central difference approximation for the second spatial derivative
- Assumes fixed boundary conditions (boundary points are excluded from computation)
- Requires `L` (domain length) and `N` (number of points) to be defined in the current scope
- The spatial step size dx = L/N is used in the finite difference formula
"""

function wave_equation(psi, c, L, N)
    # Starts from second point to second last point
    d2psi_dt2 = c^2 * (psi[1:end-2] - 2 * psi[2:end-1] + psi[3:end]) / (L / N)^2
    return d2psi_dt2
end
@benchmark wave_equation(psi_tst, 1, 1, N_test)

In [ ]:
function wave_equation_inb(psi, c, L, N)
    dx2_inv = (N / L)^2
    c2 = c^2
    d2psi_dt2 = similar(psi, length(psi) - 2)
    @inbounds for i in 1:length(d2psi_dt2)
        d2psi_dt2[i] = c2 * (psi[i] - 2 * psi[i+1] + psi[i+2]) * dx2_inv
    end
    return d2psi_dt2
end
@benchmark wave_equation_inb(psi_tst, 1, 1, N_test)

In [ ]:
function wave_equation_vec(psi, c, L, N)
    dx2_inv = (N / L)^2
    c2 = c^2
    @views @. c2 * (psi[1:end-2] - 2 * psi[2:end-1] + psi[3:end]) * dx2_inv
end
@benchmark wave_equation_vec(psi_tst, 1, 1, N_test)

In [ ]:
function wave_equation_dist(psi, c, L, N)
    n = length(psi) - 2
    d2psi_dt2 = SharedArray{Float64}(n)

    @distributed for i in 1:n
        d2psi_dt2[i] = c^2 * (psi[i] - 2 * psi[i+1] + psi[i+2]) / (L / N)^2
    end

    return Array(d2psi_dt2)
end
@benchmark wave_equation_dist(psi_tst, 1, 1, N_test)

In [ ]:
function wave_equation_simd(psi, c, L, N)
    n = length(psi) - 2
    d2psi_dt2 = Vector{Float64}(undef, n)
    dx2_inv = (N / L)^2
    c2 = c^2

    @inbounds @simd for i in 1:n
        d2psi_dt2[i] = c2 * (psi[i] - 2 * psi[i+1] + psi[i+2]) * dx2_inv
    end
    return d2psi_dt2
end
@benchmark wave_equation_simd(psi_tst, 1, 1, N_test)

In [ ]:
function wave_equation_avx(psi, c, L, N)
    n = length(psi) - 2
    d2psi_dt2 = Vector{Float64}(undef, n)
    dx2_inv = (N / L)^2
    c2 = c^2

    @inbounds @avx for i in 1:n
        d2psi_dt2[i] = c2 * (psi[i] - 2 * psi[i+1] + psi[i+2]) * dx2_inv
    end
    return d2psi_dt2
end
@benchmark wave_equation_avx(psi_tst, 1, 1, N_test)

In [ ]:
function wave_equation_dist_avx(psi, c, L, N)
    n = length(psi) - 2
    d2psi_dt2 = SharedArray{Float64}(n)
    dx2_inv = (N / L)^2
    c2 = c^2

    # Slice the array into chunks for each worker
    @sync for p in workers()
        @async begin
            @fetchfrom p begin
                local start_idx = (p - 1) * div(n, nprocs()) + 1
                local end_idx = min(p * div(n, nprocs()), n) + 1
                @inbounds @avx for i in start_idx:end_idx
                    d2psi_dt2[i] = c2 * (psi[i] - 2 * psi[i+1] + psi[i+2]) * dx2_inv
                end
            end
        end
    end

    return Array(d2psi_dt2)
end
@benchmark wave_equation_dist_avx(psi_tst, 1, 1, N_test)

In [ ]:
initial_condition_0 = (x) -> sin( x)
initial_condition_1 = (x) -> sin(2pi * x)
initial_condition_2 = (x) -> sin(5pi * x)
initial_condition_3 = (x) -> (x > 1/5 && x < 2/5) ? sin(5pi * x) : 0.0


In [ ]:
# Test finite difference approximation of the wave equation
L = 2pi
N = 100
dx = L/N
x = 0:dx:L
psi = initial_condition_0.(x)
dpsi_dt = zeros(N)
c = 1
d2psi_dt2 = wave_equation_simd(psi, c, L, N)
# Plot initial condition and second time derivative
plot(x, psi, label="Initial Condition", title="Wave Equation Test")
plot!(x[2:end-1], d2psi_dt2, label="Second Time Derivative", xlabel="x", ylabel="Value")

In [ ]:
function propagate_psi(psi_0_f::Function; L=1, N=100, c=1, t_0=0, t_f=1, dt=0.01)
    # Initial condition at t=0
    psi_x_i = psi_0_f.(range(0, L, length=N))

    # The string is at rest at t=0
    # Array is two elements shorter due to second derivative calculation
    dpsi_dt_i = zeros(N-2)

    # Boundaries are fixed to zero
    psi_x_i[1] = 0
    psi_x_i[end] = 0
    
    # Initialize the array to store the results
    n_steps = Int((t_f - t_0) / dt)
    psi_x_t = zeros(N, n_steps)
    psi_x_t[:, 1] = psi_x_i  

    # Time propagation using finite difference method
    for n in 1:n_steps-1
        d2psi_dt2 = wave_equation_avx(psi_x_t[:, n], c, L, N)
        # Use Euler's method to update psi and its time derivative
        dpsi_dt_i += d2psi_dt2 * dt
        psi_x_t[2:end-1, n+1] = psi_x_t[2:end-1, n] + dpsi_dt_i * dt
    end

    return psi_x_t
end



In [ ]:
t_0 = 0
t_f = 1
dt = 0.001
c=1
L=1
N=1000

In [ ]:
solution_1_euler = propagate_psi(initial_condition_1, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_2_euler = propagate_psi(initial_condition_2, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_3_euler = propagate_psi(initial_condition_3, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)

In [ ]:
function distributed_gif(t_f, t_0, dt, L, N, solution_1, solution_2, solution_3, gifname)
    # Animate the solution and save frames
    default(legend=false)
    i_total = Int((t_f - t_0) / dt)
    filenames = [lpad(i, 6, "0") * ".png" for i in 1:i_total]
    @sync @distributed for i in range(1, stop=i_total, length=i_total)
        i_int = Int(i)

        x = range(0, L, length=N)

        # create a plot with 3 subplots and a custom layout
        p = plot(x, solution_1[:, i_int], ylim=(-1, 1), title="Time: $(round(i_int*dt, digits=2)) s", xlabel="Position along string", ylabel="Displacement")
        plot!(x, solution_2[:, i_int])
        plot!(x, solution_3[:, i_int])
        savefig("tmp/$(lpad(i_int, 6, "0")).png")
    end

    # Create an animation from the saved frames
    anim = Animation("tmp", filenames)
    Plots.buildanimation(anim, gifname, fps=30, show_msg=false)
end

In [ ]:
distributed_gif(t_f, t_0, dt, L, N, solution_1_euler, solution_2_euler, solution_3_euler, "ex_1_anim_euler.gif")

In [ ]:
function propagate_psi_leapfrog(psi_0_f::Function; L=1, N=100, c=1, t_0=0, t_f=1, dt=0.01)
    # Initial condition at t=0
    x_i = psi_0_f.(range(0, L, length=N))

    # The string is at rest at t=0
    # Array is two elements shorter due to second derivative calculation
    v_i_slice = zeros(N - 2)

    # Boundaries are fixed to zero
    x_i[1] = 0
    x_i[end] = 0

    # Initialize the array to store the results
    psi_x_t = zeros(N, Int((t_f - t_0) / dt))
    psi_x_t[:, 1] = x_i

    # Time propagation using finite difference method
    for n in 1:size(psi_x_t, 2)-1
        # Calculate acceleration (second time derivative) at current time step
        a_i = wave_equation_avx(x_i, c, L, N)

        # Update position using Verlet integration (leapfrog)
        x_i_next_slice = x_i[2:end-1] + v_i_slice * dt + 0.5 * a_i * dt^2

        # Store the next position in the results array
        psi_x_t[2:end-1, n+1] = x_i_next_slice

        # Calculate acceleration at the next time step for velocity update
        x_i_next = psi_x_t[:, n+1]
        a_i_next = wave_equation_avx(x_i_next, c, L, N)

        # Next time step for velocity
        v_i_slice += 0.5 * (a_i + a_i_next) * dt

        # Shift current position to next for the next iteration
        x_i = x_i_next

    end

    return psi_x_t
end



In [ ]:
solution_1_leapfrog = propagate_psi_leapfrog(initial_condition_1, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_2_leapfrog = propagate_psi_leapfrog(initial_condition_2, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)
solution_3_leapfrog = propagate_psi_leapfrog(initial_condition_3, L=L, N=N, c=c, t_0=t_0, t_f=t_f, dt=dt)

In [ ]:
distributed_gif(t_f, t_0, dt, L, N, solution_1_leapfrog, solution_2_leapfrog, solution_3_leapfrog, "ex_1_anim_leapfrog.gif")

We see visibly smaller 'zig-zag' behaviour in displacement between the peaks of y3